In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
from pose_graph_partitioning.pose_graph import *
from pose_graph_partitioning.pose_graph_partitioning import *
from simulate_utils import ATE, align_posegraphs, call_d2pgo_opti
from subprocess import PIPE, Popen
import copy

In [ ]:
def call_d2pgo(input_data, output_data, agent_num, params, param_fixed={}, cmd="roslaunch d2pgo d2pgo_test_multi.launch "):
    command = f"{cmd} g2o_path:={input_data} output_path:={output_data} agent_num:={agent_num} "
    for k, v in params:
        command += f"{k}:={v[0]} "
    for k in param_fixed:
        v = param_fixed[k]
        command += f"{k}:={v} "
    p = Popen(command, shell=True, stdout=PIPE, stderr=PIPE)
    print(command)
    stdout, stderr = p.communicate()
    # return stdout, stderr
    # print(stdout.decode("utf-8"))
    return

def check_param_unique(params):
    cnt = 0
    for k, v in params:
        if len(v) > 1:
            return False, cnt
        cnt += 1
    return True, -1 

def print_res_to_table(res, sort_index="final_cost"):
    import tabulate
    output_table = []
    #Add title
    if len(res) == 0:
        return []
    sort_values = []
    for row in res:
        output_table.append([])
        for k in row:
            v = row[k]
            if isinstance(v, np.ndarray) or isinstance(v, list):
                output_table[-1].append(v[0])
            else:
                if k == "ATE_T":
                    output_table[-1].append(f"{v:.3f}m")
                elif k == "ATE_rot":
                    output_table[-1].append(f"{v*57.3:.3f}")
                elif k == "final_cost":
                    output_table[-1].append(f"{v:.3f}")
                else:
                    output_table[-1].append(v)
            if k == sort_index:
                sort_values.append(v)
    output_table = np.array(output_table)
    sort_values = np.array(sort_values)
    output_table = output_table[sort_values.argsort()]
    row0 = []
    for key in res[0]:
        row0.append(key)
    output_table = output_table.tolist()
    output_table.insert(0, row0)
    return tabulate.tabulate(output_table, tablefmt='html')

def grid_search(input_data, output_data, gt_data, agent_num, params, params_fixed={}, cmd="roslaunch d2pgo d2pgo_test_multi.launch ", is_root=True):
    #Gird search in param lists
    pg_gt = PoseGraph()
    pg_gt.read_g2o_single(gt_data)
    params_num = len(params)
    #DFS on params
    unique, row = check_param_unique(params)
    if unique:
        #Only one param, no need to grid search
        call_d2pgo(input_data, output_data, agent_num, params, params_fixed, cmd)
        pg_output = PoseGraph()
        pg_output.read_g2o_folder(output_data, prt=False)
        ate_T, ate_rot = ATE(pg_output, pg_gt)
        # align_posegraphs(pg_output, pg_gt)
        # ax = pg_gt.show("GT")
        # pg_output.show("output", ax=ax, clear=False)
        # print(params, ate_T, ate_rot)
        ret_data = {}
        for k, v in params:
            ret_data[k] = v[0]
        ret_data["ATE_T"] = ate_T
        ret_data["ATE_rot"] = ate_rot
        ret_data["final_cost"] = pg_output.evaluate_cost()
        # Add this data to output_data/result.txt
        with open(output_data + "/search_result.txt", "a") as f:
            # Save the params to a line of dict
            f.write(str(ret_data) + "\n")
        return [ret_data]
    else:
        #More than one param, need to grid search
        param_values = params[row][1]
        res = []
        for i in range(len(param_values)):
            new_params = copy.deepcopy(params)
            new_params[row][1] = [param_values[i]]
            ret = grid_search(input_data, output_data, gt_data, agent_num, new_params, params_fixed, cmd, is_root=False)
            for obj in ret:
                res.append(obj)
    return res

def random_search(input_data, output_data, agent_num, params, gt_data=None, max_trials=100, params_fixed={}, cmd="roslaunch d2pgo d2pgo_test_multi.launch ", is_root=True):
    #Gird search in param lists
    if gt_data is not None:
        pg_gt = PoseGraph()
        pg_gt.read_g2o_single(gt_data)
    #DFS on params
    res = []
    for i in range(max_trials):
        # Take a random sample of params
        ret_data = {}
        params_instance = []
        for param in params:
            params_instance.append([param[0], [np.random.choice(param[1])]])
            ret_data[param[0]] = params_instance[-1][1]
        print(f"trial {i}:", params_instance)
        #Only one param, no need to grid search
        call_d2pgo(input_data, output_data, agent_num, params_instance, params_fixed, cmd)
        pg_output = PoseGraph()
        pg_output.read_g2o_folder(output_data, prt=False)
        if gt_data is not None:
            ate_T, ate_rot = ATE(pg_output, pg_gt)
            ret_data["ATE_T"] = ate_T
            ret_data["ATE_rot"] = ate_rot
        ret_data["final_cost"] = pg_output.evaluate_cost()
        # Add this data to output_data/result.txt
        with open(output_data + "/search_result.txt", "a") as f:
            # Save the params to a line of dict
            f.write(str(ret_data) + "\n")
        res.append(ret_data)
    return res

In [ ]:
input_data = "/home/xuhao/data/d2slam/pgo/parking-garage/input"
gt_data = "/home/xuhao/data/d2slam/pgo/parking-garage/groundtruth.g2o"
output_data = "/home/xuhao/data/d2slam/pgo/parking-garage/searching-output/"
agent_num = 5
params = [
    ["max_steps", [100]],
    ["rho_frame_T", np.linspace(0.01, 2.0, 10000)],
    ["rho_frame_theta", np.linspace(0.01, 2.0, 10000)],
    ["eta_k", np.linspace(0.5, 3, 100)],
    ["enable_rot_init", ["True", "False"]]
]
ret = random_search(input_data, output_data, agent_num, params, max_trials=1000)
print_res_to_table(ret)

In [ ]:
input_data = "/home/xuhao/data/d2slam/pgo/tum_corr_5/input/"
gt_data = "/home/xuhao/data/d2slam/pgo/tum_corr_5/input/fullGraph_optimized.g2o"
output_data = "/home/xuhao/data/d2slam/pgo/tum_corr_5/searching-output/"
agent_num = 5
# params = [
#     ["max_steps", [100]],
#     ["rho_frame_T", np.logspace(-5, -1, 10000, base=10)],
#     ["rho_frame_theta", np.logspace(-2, 2, 10000, base=10)],
#     ["eta_k", np.logspace(-1, 0, 100, base=10)],
#     ["enable_rot_init", ["false", "true"]]
# ]
params = [
    ["max_steps", [10000]],
    ["rho_frame_T", np.linspace(0.01, 5.0, 10000)],
    ["rho_frame_theta", np.linspace(0.01, 5.0, 10000)],
    ["eta_k", np.linspace(0.5, 3, 100)],
    ["rho_rot_mat", np.linspace(0.001, 3, 100)],
    ["enable_rot_init", ["True"]]
]
ret = random_search(input_data, output_data, agent_num, params, max_trials=1000)
print_res_to_table(ret)

In [ ]:
#Show the convergence of the results
intput_data = "/home/xuhao/data/d2slam/pgo/torus3D/input/"
gt_data = "/home/xuhao/data/d2slam/pgo/torus3D/groundtruth.g2o"
output_data = "/home/xuhao/data/d2slam/pgo/torus3D/output/"
agent_num = 5
params = [
    ["max_steps", np.logspace(0, 2, 10, base=10, dtype=int)],
    ["debug_rot_init_only", ["true"]]
]
fixed_param = {
    "debug_rot_init_only":"true"
}
ret = grid_search(intput_data, output_data, gt_data, agent_num, params, fixed_param)
print_res_to_table(ret, "max_steps")

In [ ]:
intput_data = "/home/xuhao/data/d2slam/pgo/torus3D/input/"
gt_data = "/home/xuhao/data/d2slam/pgo/torus3D/groundtruth.g2o"
output_data = "/home/xuhao/data/d2slam/pgo/torus3D/output/"
agent_num = 5
params = [
    ["rho_rot_mat", np.logspace(-3, 1, 10, base=10)]
]
fixed_param = {
    "debug_rot_init_only":"true",
    "max_steps": 20
}
ret = grid_search(intput_data, output_data, gt_data, agent_num, params, fixed_param)
print_res_to_table(ret, "ATE_rot")

In [ ]:
def search_function(param):
    pg_output, _, _ = call_d2pgo_opti(input_data, output_data, agent_num = agent_num, max_steps=100, enable_rot_init=True, 
        eta_k=param['eta_k'], rho_frame_theta=param['rho_frame_theta'], rho_frame_T=param['rho_frame_T'])
    return pg_output.evaluate_cost()
# input_data = "/home/xuhao/data/d2slam/pgo/tum_corr_5/input"
# output_data = "/home/xuhao/data/d2slam/pgo/tum_corr_5/searching-output/"

input_data = "/home/xuhao/data/d2slam/pgo/parking-garage/input"
output_data = "/home/xuhao/data/d2slam/pgo/parking-garage/searching-output/"

agent_num = 5
from hyperopt import hp, fmin, tpe
space = {
    "rho_frame_T": hp.uniform("rho_frame_T", 0.0, 10.0),
    "rho_frame_theta": hp.uniform("rho_frame_theta", 0.0, 10.0),
    "eta_k": hp.uniform("eta_k", 0, 5.0),
}

best = fmin(search_function, space, algo=tpe.suggest, max_evals=100)
print(best)